# Retraining YOLO for Screen Time Detection

## Create Training/Testing Set 

In [7]:
import glob as glob
import os
from pathlib import Path

import cv2
import numpy as np
import pandas as pd
from sklearn.metrics import (
    classification_report,
    cohen_kappa_score,
    confusion_matrix,
    roc_auc_score,
)

import image_processor

### Get Classes and Manually Labelled Data
We use the classes that we have used in the manual labelling as our target set of classes.

In [ ]:
# TODO
# Wrangle labelled data

### Get Pre-Labelled Data for Relevant Classes
We can supplement our training data using the COCO dataset for classes that are in our target classes. If the model is still weak in some areas, we can try supplementing this with more pre-labelled datasets.

In [1]:
# TODO
# Check if COCO data is already downloaded. If not, download the labels and images that we want.

In [2]:
# TODO
# Combine the manually labelled and the COCO datasets

### Create Training Split
We could normally just define a random split. But, we want to avoid a couple of potential issues. So, our training split should be random but follow some rules:
1. COCO images should be training and testing only, but shouldn't be included in the validation set.
2. Similiarly, if a participant is in the training set, they should not be in the validation set. Many images end up being far too similar, and we risk not really understanding how the model will perform on 'unseen' data.

In [ ]:
# TODO
# Create training/testing/validation sets

## Retrain the Model

In [3]:
# TODO
# Retrain model and save new model weights

## Evaluate the Model

In [ ]:
# TODO
# Evaluate the model on labelled data.